<a href="https://colab.research.google.com/github/NourEldin-Osama/Python-Projects/blob/main/ACO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# implement Ant Colony Optimization
from itertools import accumulate
from random import random
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# initialize parameters
num_iterations = 2
lower_bound, upper_bound = 1, 4
num_ants = 4
step_size = 1
pheromone = 1
scaling_factor = 2
evoporation_rate = 0.5

In [3]:
# Load the training data.
data = pd.read_csv('sample_data/california_housing_train.csv')
x_train = data.drop(columns = ['median_house_value'])
y_train = data['median_house_value']

# Load the test data.
data = pd.read_csv('sample_data/california_housing_train.csv')
x_test = data.drop(columns = ['median_house_value'])
y_test = data['median_house_value']

In [4]:
# generate range of values
def generate_range(lower_bound, upper_bound, step_size):
    x = []
    while lower_bound <= upper_bound:
        x.append(lower_bound)
        lower_bound += step_size
    return x

# get the index of the first value greater than r
def get_x_index(r, cumulative_probabilities):
    for i in range(len(cumulative_probabilities)):
        if cumulative_probabilities[i] > r:
            return i

def updated_pheromone(old_pheromone):
    return (1-evoporation_rate)*old_pheromone

In [5]:
def create_model(num_hidden_layers):
    # Define the input layer
    inputs = keras.Input(shape=(8,))
    
    # Add the hidden layers
    x = inputs
    for i in range(num_hidden_layers):
        x = layers.Dense(32, activation="relu")(x)
    
    # Add the output layer
    outputs = layers.Dense(1, activation="linear")(x)
    
    # Create the model
    model = keras.Model(inputs=inputs, outputs=outputs, name="regression_model")
    
    # Compile the model
    model.compile(loss="mse", optimizer="adam")
    return model

In [6]:
def objective_function(num_hidden_layers):
    # Create the model with 2 hidden layers
    model = create_model(num_hidden_layers)

    # Train the model on some data
    history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
    loss = history.history['loss'][-1]
    
    return loss

In [7]:
xs = generate_range(lower_bound, upper_bound, step_size)
m = len(xs)
print(f"{xs = }", f"{m = }")
# generate an initial pheromone list
pheromones = [pheromone for i in range(m)]
for iteration in range(num_iterations):
    print(f"iteration {iteration + 1}")
    sum_pheromones = sum(pheromones)
    print(f"{pheromones = }", f"{sum_pheromones = }")
    probabilities = [pheromones[i]/sum_pheromones for i in range(m)]
    print(f"{probabilities = }")
    cumulative_probabilities = list(accumulate(probabilities))
    cumulative_probabilities = [round(value,4) for value in cumulative_probabilities]
    print(f"{cumulative_probabilities = }")
    # generate random numbers
    rs = [random() for i in range(num_ants)]
    print(f"{rs = }")
    rs_xs = [get_x_index(r, cumulative_probabilities) for r in rs]
    print(f"{rs_xs = }")
    results = [objective_function(xs[x]) for x in rs_xs]
    print(f"{results = }")

    # get the best result, and the best x, the worst result
    # minimize the objective function
    best_result = min(results)
    best_x = rs_xs[results.index(best_result)]
    worst_result = max(results)
    print(f"{best_result = }", f"{best_x = }", f"{worst_result = }")

    # update the best pheromone
    pheromones[best_x] = pheromones[best_x] + scaling_factor*(best_result/worst_result)

    # update other pheromones
    pheromones[:best_x] = [updated_pheromone(worst_pheromone) for worst_pheromone in pheromones[:best_x]]
    pheromones[best_x+1:] = [updated_pheromone(worst_pheromone) for worst_pheromone in pheromones[best_x+1:]]
    print(f"{pheromones = }")

    print(f"{best_result = }")
    print(f"Best value of x = {xs[best_x]}")
    print()

xs = [1, 2, 3, 4] m = 4
iteration 1
pheromones = [1, 1, 1, 1] sum_pheromones = 4
probabilities = [0.25, 0.25, 0.25, 0.25]
cumulative_probabilities = [0.25, 0.5, 0.75, 1.0]
rs = [0.26567466760728053, 0.9209349928377158, 0.46729686167439843, 0.47779808070774554]
rs_xs = [1, 3, 1, 1]
Epoch 1/10
532/532 [==============================] - 4s 6ms/step - loss: 35520294912.0000 - val_loss: 26549135360.0000
Epoch 2/10
532/532 [==============================] - 3s 5ms/step - loss: 25707223040.0000 - val_loss: 24508721152.0000
Epoch 3/10
532/532 [==============================] - 3s 6ms/step - loss: 22796115968.0000 - val_loss: 20708458496.0000
Epoch 4/10
532/532 [==============================] - 3s 6ms/step - loss: 18232201216.0000 - val_loss: 15635778560.0000
Epoch 5/10
532/532 [==============================] - 3s 5ms/step - loss: 13726423040.0000 - val_loss: 12214835200.0000
Epoch 6/10
532/532 [==============================] - 2s 3ms/step - loss: 11647618048.0000 - val_loss: 11299717120.000